In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Model

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import scipy
import os
import random
import gc
from keras.callbacks import ModelCheckpoint
from zipfile import ZipFile
import glob

2022-04-22 00:07:33.339701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 00:07:33.339745: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#Checking corrupt images 2nd method.
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
path = Path("/com.docker.devenvironments.code/fer2013/trainimgs").rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

KeyboardInterrupt: 

In [ ]:
training = ImageDataGenerator(rescale=1./255)
testing = ImageDataGenerator(rescale=1./255)

#Increase batch size for faster training/ learns less per epoch.

traindata = training.flow_from_directory('/com.docker.devenvironments.code/fer2013/trainimgs',
target_size=(48,48),
batch_size=128,
color_mode='rgb',
class_mode='categorical')

testdata = testing.flow_from_directory('/com.docker.devenvironments.code/fer2013/testimgs',
target_size=(48,48),
batch_size=128,
color_mode='rgb',
class_mode='categorical')

In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape=(48,48,3), include_top=False, weights='imagenet')

In [ ]:
#ResNet model with additional convolutional layers.
model = Sequential()
model.add(base_model)
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(48,48,3), data_format='channels_last'))
model.add(Conv2D(64,kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(128,kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Conv2D(128,kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='sigmoid'))


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model_info = model.fit(traindata,epochs=100,validation_data=testdata)


#base_model.compile(loss='categorical_crossentrophy', optimizer='adam', metrics=['accuracy'])
#model_info = base_model.fit(traindata, steps_per_epoch=449,epochs=100,validation_data=testdata,validation_steps=112)


In [ ]:
model.save('ResNet50_Improved')
new = tf.keras.models.load_model('ResNet50_Improved')
#model.evaluate()
model.summary()


In [ ]:
#ResNet model without any additional layers.
resnet = Sequential()
resnet.add(base_model)
resnet.add(Flatten())
resnet.add(Dense(200, activation='relu'))
resnet.add(Dense(7, activation='softmax'))
resnet.summary()

resnet.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
resnet.fit(traindata,epochs=100,validation_data=testdata)
resnet.save('ResNet50_Basic')